In [1]:
import pandas as pd
import numpy as np

In [2]:
#export_au 파일 불러오기
export_au=pd.read_table('Trade_Map_-_List_of_products_exported_by_Australia.txt', sep='\t')
export_au.rename(columns={'Code':'HS'}, inplace=True)
export_au.drop(export_au.index[0], inplace=True)
export_au['HS']=export_au['HS'].astype('int64')

In [3]:
#HS17 코드 불러와서 export_au 파일과 병합
hs_bec=pd.read_csv('hs_bec17.csv')
export_au=pd.merge(export_au, hs_bec, how='left')

In [4]:
#최근 10년간 거래가 없는 품목 제외시킴
no_amount_au=export_au[(export_au['Exported value in 2012']==0)
                       & (export_au['Exported value in 2013']==0) 
                       & (export_au['Exported value in 2014']==0) 
                       & (export_au['Exported value in 2015']==0) 
                       & (export_au['Exported value in 2016']==0) 
                       & (export_au['Exported value in 2017']==0)  
                       & (export_au['Exported value in 2018']==0) 
                       & (export_au['Exported value in 2019']==0) 
                       & (export_au['Exported value in 2020']==0) 
                       & (export_au['Exported value in 2021']==0)].index
export_au.drop(no_amount_au, inplace=True)

In [5]:
#HS 999999품목은 따로 묶기 위해 임의의 BEC 번호 999999를 만들어 줌
export_au.loc[export_au['HS'] == 999999, 'BEC17'] = 999999

In [6]:
#BEC17 결측치 확인
export_au[export_au['BEC17'].isnull()]

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,BEC17
4670,30627,"Shrimps and prawns, even smoked, whether in sh...",446,32,125,9,3,0,0,0,0,0,NaN,NaN
4672,30789,"Smoked, frozen, dried, salted or in brine, aba...",55228,58161,49250,48881,49226,0,0,0,0,0,NaN,NaN
4718,440310,"Wood in the rough, treated with paint, stains,...",6377,4912,2826,3044,3873,0,0,0,0,0,NaN,NaN
4748,845610,Machine tools for working any material by remo...,3104,3194,1063,1132,3134,0,0,0,0,0,NaN,NaN
4764,900610,Cameras of a kind used for preparing printing ...,289,56,881,273,163,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6028,401194,"Pneumatic tyres, new, of rubber, of a kind use...",4954,6683,3865,5226,4370,0,0,0,0,0,NaN,NaN
6053,690790,"Unglazed ceramic flags and paving, hearth or w...",5558,4040,3675,2789,1994,0,0,0,0,0,NaN,NaN
6054,690890,"Glazed ceramic flags and paving, hearth or wal...",2234,1698,2057,1609,1349,0,0,0,0,0,NaN,NaN
6075,852851,Monitors of a kind solely or principally used ...,32586,33001,34558,34220,15801,0,0,0,0,0,NaN,NaN


In [7]:
#BEC17 결측치 있는 행들만 모아 데이터 프레임을 만듬(아래에서 BEC12 파일과 병합을 위해)
BEC17_nan_au=export_au[export_au['BEC17'].isnull()]

In [8]:
#BEC12 파일과 BEC17 결측치가 담긴 데이터프레임을 병합
hs_bec12=pd.read_csv('hs_bec12.csv')
BEC17_nan_au=pd.merge(BEC17_nan_au, hs_bec12, how='left')

In [9]:
#BEC12와 모두 매치 되어 결측치 없음을 확인
BEC17_nan_au.isnull().sum()

HS                         0
Product label              0
Exported value in 2012     0
Exported value in 2013     0
Exported value in 2014     0
Exported value in 2015     0
Exported value in 2016     0
Exported value in 2017     0
Exported value in 2018     0
Exported value in 2019     0
Exported value in 2020     0
Exported value in 2021     0
Unnamed: 12               78
BEC17                     78
BEC12                      0
dtype: int64

In [10]:
#BEC17로 채워진 행들(export_au)과 BEC12(BEC17_nan_au)로 채워진 행들을 합치기
export_au_complete=pd.merge(export_au, BEC17_nan_au, how='left')

In [11]:
#컬럼들을 보기 좋게 배열하기 위해 컬럼명 확인
export_au_complete.columns

Index(['HS', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12', 'BEC17', 'BEC12'],
      dtype='object')

In [12]:
export_au_complete=export_au_complete[['HS', 'BEC17', 'BEC12', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]


In [13]:
export_au_complete

,HS,BEC17,BEC12,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,260111,2111.0,NaN,Non-agglomerated iron ores and concentrates (e...,59192644,68522713,69260622,36622883,39226993,49108087,46529095,65588606,78020629,115981729,NaN
1,270112,2111.0,NaN,"Bituminous coal, whether or not pulverised, no...",43691288,38380799,35187114,29623822,28260420,40555876,47206109,44407758,32725070,43934284,NaN
2,271111,213101.0,NaN,"Natural gas, liquefied",14676410,14824997,16451316,12582034,13559824,19937263,31970665,35167504,26312442,36957140,NaN
3,710813,411210.0,NaN,"Gold, incl. gold plated with platinum, in semi...",15141407,12938150,11889298,10709989,13390949,12999194,13986349,16124214,15983700,15003130,NaN
4,999999,999999.0,NaN,Commodities not elsewhere specified,10044824,8777678,8032952,7321442,7391488,8467782,7940698,7904342,8017722,11432602,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222,690790,NaN,311220.0,"Unglazed ceramic flags and paving, hearth or w...",5558,4040,3675,2789,1994,0,0,0,0,0,NaN
5223,690890,NaN,311220.0,"Glazed ceramic flags and paving, hearth or wal...",2234,1698,2057,1609,1349,0,0,0,0,0,NaN
5224,844530,412010.0,NaN,Textile doubling or twisting machines,51,269,144,292,0,0,0,88,0,0,NaN
5225,852851,NaN,613202.0,Monitors of a kind solely or principally used ...,32586,33001,34558,34220,15801,0,0,0,0,0,NaN


In [14]:
#BEC17 컬럼과 BEC12 컬럼을 합쳐주기 위해 결측치를 0으로 처리
export_au_complete['BEC17']=export_au_complete['BEC17'].fillna(0)
export_au_complete['BEC12']=export_au_complete['BEC12'].fillna(0)
#BEC17 컬럼과 BEC12 컬럼의 값을 더해서 BEC 컬럼을 만들고 기존의 BEC17 컬럼과 BEC12 컬럼은 제외
export_au_complete['BEC']=export_au_complete['BEC17']+export_au_complete['BEC12']
export_au_complete=export_au_complete.drop(columns=['BEC17','BEC12'])

In [15]:
export_au_complete=export_au_complete[['HS', 'BEC','Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]

In [16]:
export_au_complete

,HS,BEC,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,260111,2111.0,Non-agglomerated iron ores and concentrates (e...,59192644,68522713,69260622,36622883,39226993,49108087,46529095,65588606,78020629,115981729,NaN
1,270112,2111.0,"Bituminous coal, whether or not pulverised, no...",43691288,38380799,35187114,29623822,28260420,40555876,47206109,44407758,32725070,43934284,NaN
2,271111,213101.0,"Natural gas, liquefied",14676410,14824997,16451316,12582034,13559824,19937263,31970665,35167504,26312442,36957140,NaN
3,710813,411210.0,"Gold, incl. gold plated with platinum, in semi...",15141407,12938150,11889298,10709989,13390949,12999194,13986349,16124214,15983700,15003130,NaN
4,999999,999999.0,Commodities not elsewhere specified,10044824,8777678,8032952,7321442,7391488,8467782,7940698,7904342,8017722,11432602,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222,690790,311220.0,"Unglazed ceramic flags and paving, hearth or w...",5558,4040,3675,2789,1994,0,0,0,0,0,NaN
5223,690890,311220.0,"Glazed ceramic flags and paving, hearth or wal...",2234,1698,2057,1609,1349,0,0,0,0,0,NaN
5224,844530,412010.0,Textile doubling or twisting machines,51,269,144,292,0,0,0,88,0,0,NaN
5225,852851,613202.0,Monitors of a kind solely or principally used ...,32586,33001,34558,34220,15801,0,0,0,0,0,NaN


In [17]:
export_au_complete.to_csv("export_au.csv", index = False)